In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import random
from scipy import stats

In [4]:
wa_data = pd.read_csv('../Inputs/SALESInputs/WA_MJ_Sales_and_Taxes.csv')
wa_data = wa_data[:-1]
wa_data = wa_data.rename(columns={
    'Month of Activity2': 'Month',
    'Taxable retail sales as reported by marijuana collectives (before tax)3': 'Taxable Retail Sales',
    'Estimated retail value of marijuana and marijuana products sold at a licensed marijuana retailer (before tax)4': 'Estimated Retail Value',
    'Total retail value of marijuana and marijuana products (before tax)': 'Total Retail Value',
    'Sales exempt from sales tax for medical purposes5': 'Tax-exempt Sales',
    'Estimated retail value of marijuana and marijuana products subject to state and local sales taxes': 'Est. Retail Value subject to Tax',
    'Estimated state sales tax collections': 'Est. State Sales Tax Collections',
    'Weighted average local sales tax rate6': 'Avg. Local Sales Tax Rate',
    'Estimated local sales tax collections': 'Est. Local Sales Tax Collections'
})
wa_data['State'] = 'WA'
wa_data

,Month,Taxable Retail Sales,Estimated Retail Value,Total Retail Value,Tax-exempt Sales,Est. Retail Value subject to Tax,Est. State Sales Tax Collections,Avg. Local Sales Tax Rate,Est. Local Sales Tax Collections,State
0,Jul-14,7478000.0,1847000.0,9325000.0,0.0,9325000.0,606000.0,0.024696,230000.0,WA
1,Aug-14,7347000.0,4555000.0,11902000.0,0.0,11902000.0,774000.0,0.024696,294000.0,WA
2,Sep-14,8598000.0,5806000.0,14404000.0,0.0,14404000.0,936000.0,0.024696,356000.0,WA
3,Oct-14,7597000.0,7746000.0,15344000.0,0.0,15344000.0,997000.0,0.024696,379000.0,WA
4,Nov-14,7526000.0,9091000.0,16618000.0,0.0,16618000.0,1080000.0,0.024696,410000.0,WA
5,Dec-14,12405000.0,11605000.0,24010000.0,0.0,24010000.0,1561000.0,0.024696,593000.0,WA
6,Jan-15,10237000.0,13096000.0,23334000.0,0.0,23334000.0,1517000.0,0.024696,576000.0,WA
7,Feb-15,9869000.0,16086000.0,25955000.0,0.0,25955000.0,1687000.0,0.024696,641000.0,WA
8,Mar-15,11367000.0,21363000.0,32730000.0,0.0,32730000.0,2127000.0,0.024696,808000.0,WA
9,Apr-15,11451000.0,24854000.0,36306000.0,0.0,36306000.0,2360000.0,0.024696,897000.0,WA


In [5]:
wa_total_retail_value = wa_data['Total Retail Value'].sum()
wa_total_retail_value

3046177000.0

In [6]:
co_data = pd.read_csv('../Inputs/SALESInputs/CO_MJ_Sales.csv')
co_data['State'] = 'CO'
co_data = co_data.drop(columns=[
    'Medical Monthly Sales',
    'Medical YTD'
])
co_data

,Month,Retail Monthly Sales,Retail YTD,Total Monthly Sales,Total YTD,Total to Date,State
0,Jan 2014,14022213,14022213,46563933,46563933,46563933,CO
1,Feb 2014,14248473,28270686,45987045,92550978,92550978,CO
2,Mar 2014,19881631,48152317,54703509,147254487,147254487,CO
3,Apr 2014,20765986,68918303,53452855,200707342,200707342,CO
4,May 2014,21375001,90293304,52730209,253437551,253437551,CO
5,Jun 2014,23978082,114271386,53928391,307365942,307365942,CO
6,Jul 2014,29866792,144138178,61004415,368370357,368370357,CO
7,Aug 2014,33520608,177658786,67432834,435803191,435803191,CO
8,Sep 2014,30345357,208004143,63066595,498869786,498869786,CO
9,Oct 2014,31185985,239190128,62965201,561834987,561834987,CO


In [7]:
ak_data = pd.read_csv('../Inputs/SALESInputs/AK_MJ_Sales.csv')
ak_data = ak_data[:-1]
ak_data

,Month,Year,Taxpayers,Cultivator Tax Liability,Tax Liability to Date,Buds(lbs),Trim(lbs),Buds Sales,Trim Sales,Total Sales,Effective Tax Rate
0,October,2016.0,4.0,10400.0,10400.0,NaN,NaN,NaN,NaN,NaN,NaN
1,November,2016.0,7.0,81100.0,91500.0,NaN,NaN,NaN,NaN,NaN,NaN
2,December,2016.0,13.0,145800.0,237300.0,167.0,46.0,793584.0,36800.0,830384.0,0.175581
3,January,2017.0,17.0,107500.0,344800.0,111.0,77.0,527472.0,61600.0,589072.0,0.182490
4,February,2017.0,20.0,128000.0,472800.0,135.0,85.0,641520.0,68000.0,709520.0,0.180404
5,March,2017.0,27.0,220200.0,693000.0,225.0,169.0,1069200.0,135200.0,1204400.0,0.182830
6,April,2017.0,29.0,272000.0,965000.0,269.0,236.0,1278288.0,188800.0,1467088.0,0.185401
7,May,2017.0,31.0,272600.0,1237600.0,290.0,170.0,1378080.0,136000.0,1514080.0,0.180043
8,June,2017.0,40.0,512500.0,1750100.0,547.0,313.0,2599344.0,250400.0,2849744.0,0.179841
9,July,2017.0,46.0,577901.0,2328001.0,612.0,369.0,2908224.0,295200.0,3203424.0,0.180401


In [8]:
ore_data = pd.read_csv('../Inputs/SALESInputs/OR_MJ_Sales.csv')
ore_data = ore_data.drop(columns=['Unnamed: 3'])
ore_data 

,Month,State Tax Received,Total Sales
0,February-16,2484170,14612764.71
1,March-16,4358754,25639729.41
2,April-16,3735111,21971241.18
3,May-16,4339440,25526117.65
4,June-16,5735508,33738282.35
5,July-16,5533949,32552641.18
6,August-16,7345281,43207535.29
7,September-16,6679585,39291676.47
8,October-16,7831157,46065629.41
9,November-16,6463877,38022805.88
